# 协程
- 参考资料
# 迭代器
- 可迭代（iterable）：直接作用于for循环的变量
- 迭代器（iterator）：不但可以作用于for循环，还可以被next调用
- list是典型的可迭代对象，但不是迭代器
- 通过isinstance判断
- iterable和iterator可以互相转换
  - 通过iter函数

In [3]:
# 可迭代

l = [i for i in range(10)]

for idx in l:
    print(idx)
    
    
for i in range(5):
    print(i)

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4


In [4]:
from collections import Iterable

l = [1,2,3,4,5]

print(isinstance(l,Iterable))


from collections import Iterator

l = [1,2,3,4,5]

print(isinstance(l,Iterator))

True
False


# 生成器
- generator：一边循环一边计算下一个元素的机制/算法
- 需要满足三个条件
  - 每次调用都生成for循环需要的下一个元素或者
  - 如果达到最后一个，报出stopiteration异常
  - 可以被next函数调用
- 如何生成一个生成器
  - 直接使用
  - 如果函数包含yield，则这个函数就叫生成器
  - next调用函数，遇到yield返回

In [6]:
#直接使用生成器
l = [x*x for x in range(5)]#放在中括号就是里列表生成器
g = (x*x for x in range(5))#放在小括号就是生成器

print(type(l))
print(type(g))

<class 'list'>
<class 'generator'>


In [7]:
# 函数案例

def odd():
    print("Step 1")
    print("Step 2")
    print("Step 3")
    return None
odd()

Step 1
Step 2
Step 3


In [27]:
# 生成器案例
# 在函数odd中，yield负责返回
def odd():
    print("Step 1")
    yield 1
    print("Step 2")
    yield 2
    print("Step 3")
    yield 3
    
# odd()是调用生成器
g = odd()
one = next(g)
print(one)
two = next(g)
print(two)

three = next(g)
print(three)

Step 1
1
Step 2
2
Step 3
3


In [29]:
# for循环调用生成器

def fib(max):
    n,a,b = 0, 0, 1
    while n < max:
        print(b)
        a,b = b,a + b
        n += 1
    return "done"

fib(5)

1
1
2
3
5


'done'

In [38]:
# 斐波那契数列生成器写法

def fib(max):
    n,a,b = 0, 0, 1
    while n < max:
        print(b)
        a,b = b,a + b
        n += 1
# 需要注意，报出异常的返回值是return的返回值
    return "done"

g = fib(5)

for i in range(6):
    rst = next(g)
    print(rst)

1
1
2
3
5


TypeError: 'str' object is not an iterator

In [42]:
# 在for循环里使用生成器
# 生成器的典型用法就是在for循环使用
#比较典型的生成器是range
ge = fib(10)

for i in ge:
    print(i)

1
1
2
3
5
8
13
21
34
55
d
o
n
e


# 协程
- 历史
  - 3.4引入协程，用yield实现
  - 3.5引入协程语法
  - 实现协程比较好的包有asyncio，tornado，gevent
- 定义：协程，是为非抢占式多任务产生子程序的计算机程序组件，协程允许不同入口点在不同位置暂停或者开始执行程序
- 从技术角度，协程就是一个你可以暂停执行的函数，或者干脆把协程理解为生成器
- 协程的实现
  - yield返回
  - send调用
  
- 协程的四个状态
  - inspect.getgeneratorstate()函数确认，该函数会返回下述字符串中的一个
  - GEN_GREATED:等待开始执行
  - GEN_RUNNING:解释器正在执行
  - GEN_SUSPEND:再yield表达式处暂停
  - GEN_CLOSED:执行结束
  - next预激（prime）
- 协程终止
  - 协程中未处理的异常会向上冒泡，传给next函数或者send方法的调用方（即触发协程的对象）
  - 终止协程的一种方法：发送某个哨兵值，让协程退出，内置的None和Ellipsis等常量经常用作哨兵值==。
  
- yield from
  - 调用协程为了得到返回值，协程必须正常终止
  - 生成器正常终止会发出stopiteration异常，异常对象的value属性保存返回值。
  - yield from从内部捕获stopiteration异常
  - 委派生成器
    - 包含yield from表达式的生成器函数
    - 委派生成器在yield from表达式处暂停，调用方可以直接把数据发给生成器
    - 子生成器在把产生的值发给调用方
    - 子生成器在最后，解释器会抛出stopiteration，并把返回值附加到异常对象上
    

In [44]:
#协程案例

def simple_coroutine():
    print('-->start')
    x = yield
    print('-->recived',x)
#主线程    
sc = simple_coroutine()
print(1111)
#可以使用sc.send（None），效果一样
next(sc)#预激

print(2222)

sc.send('zhexiao')

1111
-->start
2222
-->recived zhexiao


StopIteration: 

In [45]:
#协程的状态
def simple_coroutine(a):
    print('-->start')
    b = yield a
    print('-->recived',a,b)
    
    c = yield a + b
    print('-->recived',a,b,c)
#runc   
sc = simple_coroutine(5)

aa = next(sc)
print(aa)
bb = sc.send(6)#5,6
print(bb)
cc = sc.send(7)
print(cc)#5,6,7

-->start
5
-->recived 5 6
11
-->recived 5 6 7


StopIteration: 

In [48]:
def gen():
    for c in 'AB':
        yield c
print(list(gen()))

def new_gen():
    yield from 'AB'
print(list(new_gen()))

['A', 'B']
['A', 'B']


# asyncio
- python3.4开始引入标准库当中，内置对异步io的支持
- asyncio本身是一个消息循环
- 步骤
  - 创建消息循环
  - 把协程导入
  - 关闭

# async and await
- 为了跟好的表示异步io
- python3.5引入
- 让协程代码更简洁
- 使用上，可以简单的进行替换
  - 用async替换@asyncio.coroutine
  - await替换yield from

# aiohttp
- aiohttp实现单线程的并发io，在客户端用处不大
- 在服务器端可以asyncio+coroutin配合，因为http是io操作
- asyncio实现了tcp，udp，ssl等协议
- aiohttp是给予asyncio实现的http框架

# concurrent.futures
- python3新增加的库
- 类似其他语言的线程池的概念
- 利用multiprecessiong实现真正的并行计算
- 核心原理：以子进程的形式，并行运行多个python解释器，从而令python程序可以利用多核CPU来提升执行速度
- 由于子进程和主解释器相分离，所以他们的全局解释器锁也是相互独立的
- 每个子进程都能够完整的使用一个CPU内核
- concurrent.futures.executor
  - ThreadPoolExecutor
  - ProcessPoolExecutor
  - 执行的时候需要自行选择
- submit（fn,args,kwargs）
  - fn：异步执行的函数
  - args,kwargs参数
